In [ ]:
# -- Crear los prompts (v1, v2, v3) --
prompt_v1_content = """
Human: Eres un asistente de viajes. Por favor, genera un itinerario de 3 días para un viaje a {ciudad}. El viajero está interesado en la {interes}.

Assistant:
"""
with open("prompt_template_v1.txt", "w") as f:
    f.write(prompt_v1_content)

prompt_v2_content = """
Human: Actúa como un experto guía turístico. Tu tarea es diseñar un itinerario detallado y enriquecedor de 3 días en {ciudad}. El enfoque principal del viaje debe ser {interes}. Por favor, para cada día, incluye al menos una sugerencia de restaurante local y un consejo práctico de transporte.

Assistant:
"""
with open("prompt_template_v2.txt", "w") as f:
    f.write(prompt_v2_content)

prompt_v3_content = """
Human: Tu rol es ser un planificador de viajes eficiente para un blog. Genera un itinerario de 3 días para {ciudad} enfocado en {interes}. La salida debe ser únicamente una tabla en formato Markdown con las columnas: "Día", "Actividad Principal (Enfoque en {interes})", y "Sugerencia de Comida".

Assistant:
"""
with open("prompt_template_v3.txt", "w") as f:
    f.write(prompt_v3_content)

# -- Crear el archivo Python con la lógica del modelo de Bedrock --
bedrock_model_code = """
import mlflow
import mlflow.pyfunc
import boto3
import json
import os

class BedrockPromptModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        self.bedrock_runtime = None
        prompt_path = context.artifacts["prompt_template"]
        with open(prompt_path, "r") as f:
            self.prompt_template = f.read()

    def _initialize_client(self):
        if self.bedrock_runtime is None:
            # Apuntando a la región de Ohio
            self.bedrock_runtime = boto3.client(
                service_name="bedrock-runtime",
                region_name="us-east-2"
            )

    def _generate_response(self, prompt):
        self._initialize_client()
        
        provisioned_model_arn = "arn:aws:bedrock:us-east-2:718199867798:inference-profile/us.anthropic.claude-3-5-sonnet-20241022-v2:0" 
        
        # El formato del cuerpo para los modelos Claude 3
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2048,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}]
                }
            ]
        })

        response = self.bedrock_runtime.invoke_model(
            body=body, 
            modelId=provisioned_model_arn, # Usamos el ARN aquí
            accept="application/json", 
            contentType="application/json"
        )
        
        response_body = json.loads(response.get("body").read())
        return response_body.get("content")[0].get("text")

    def predict(self, context, model_input):
        outputs = []
        for index, row in model_input.iterrows():
            prompt_formateado = self.prompt_template.format(
                ciudad=row["ciudad"], interes=row["interes"]
            )
            respuesta = self._generate_response(prompt_formateado)
            outputs.append(respuesta)
        return outputs
"""
with open("bedrock_prompt_model.py", "w") as f:
    f.write(bedrock_model_code)

print("✅ Todos los archivos necesarios han sido creados usando la lógica para un modelo aprovisionado.")

In [ ]:
import mlflow
from bedrock_prompt_model import BedrockPromptModel

# --- Configuración ---
tracking_server_arn = "arn:aws:sagemaker:us-east-2:718199867798:mlflow-tracking-server/PruebaMlflow"
mlflow.set_tracking_uri(tracking_server_arn)

experiment_name = "Experimentos_con_Prompts_de_Viajes_SageMaker"
model_name = "Generador-Itinerarios-Bedrock" 
mlflow.set_experiment(experiment_name)

def register_prompt_version(prompt_file_path, version_tag):
    with mlflow.start_run() as run:
        print(f"Registrando versión con el prompt: {prompt_file_path}")
        artifacts = {"prompt_template": prompt_file_path}
        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=BedrockPromptModel(),
            artifacts=artifacts,
            registered_model_name=model_name,
        )
        mlflow.set_tag("prompt_version", version_tag)
        mlflow.set_tag("bedrock_model", "claude-3.5-sonnet-provisioned")
    print(f"✅ Versión '{version_tag}' registrada exitosamente!")

# --- Registrar todas las versiones ---
register_prompt_version("prompt_template_v1.txt", "v1.0 - inicial")
register_prompt_version("prompt_template_v2.txt", "v2.0 - detallado")
register_prompt_version("prompt_template_v3.txt", "v3.0 - tabla markdown")

print("\n🎉 Todas las versiones del modelo han sido registradas.")

In [ ]:
import mlflow
import pandas as pd
import json

tracking_server_arn = "arn:aws:sagemaker:us-east-2:718199867798:mlflow-tracking-server/PruebaMlflow"
mlflow.set_tracking_uri(tracking_server_arn)

model_name = "Generador-Itinerarios-Bedrock"

# Se crea un nuevo experimento para guardar los resultados de las pruebas
mlflow.set_experiment("Resultados_de_Inferencia")

# Se inicia un "run"
with mlflow.start_run(run_name="Prueba_Manizales_Cafe"):
    
    # --- Datos de Entrada ---
    input_data_dict = {
        "ciudad": ["Manizales"],
        "interes": ["café y paisajes de montaña"]
    }
    input_data = pd.DataFrame(input_data_dict)
    
    # guarda la entrada como un artefacto JSON
    mlflow.log_dict(input_data_dict, "input_data.json")

    # --- Cargar y probar la Versión 2 ---
    print("Cargando el modelo Versión 2...")
    model_v2_uri = f"models:/{model_name}/2"
    loaded_model_v2 = mlflow.pyfunc.load_model(model_v2_uri)
    result_v2 = loaded_model_v2.predict(input_data)
    
    # Se guarda la salida de la v2
    mlflow.log_text(result_v2[0], "output_v2.txt")

    print("\n" + "="*60)
    print(" V2 | Respuesta de la Versión 2 (Prompt detallado):")
    print("="*60)
    print(result_v2[0])

    # --- Cargar y probar la Versión 3 ---
    print("\nCargando el modelo Versión 3...")
    model_v3_uri = f"models:/{model_name}/3" # ⚠️ Ajusta el número si es necesario
    loaded_model_v3 = mlflow.pyfunc.load_model(model_v3_uri)
    result_v3 = loaded_model_v3.predict(input_data)
    
    # 5. Se guarda la salida de la v3 como un artefacto de texto
    mlflow.log_text(result_v3[0], "output_v3.txt")

    print("\n" + "="*60)
    print(" V3 | Respuesta de la Versión 3 (Prompt de tabla):")
    print("="*60)
    print(result_v3[0])

print("\n✅ Ejecución finalizada. Revisa el experimento 'Resultados_de_Inferencia' en MLflow.")